In [29]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender,all_pairs_knn)
from sklearn.preprocessing import normalize
import numpy as np
import scipy.sparse as sp
import pickle

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks
if evaluation:
    filesslim = "./DumpData/sim_bpr_10e_100k2.npz" # or "./DumpData/sim_bpr_10e_100k.npz"
else:
    filesslim = "./DumpData/sim_bpr_10e_100k_fulltrain.npz"
    

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [30]:
#track-album
icm = dr.getICM_csr(albums=True,artists=False,tags=False)
#icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
s.setdiag(0)
r_al = urm*s.T
r_al = normalize(r_al, axis=0)
if evaluation: print dr.evaluateMAP(r_al,verbose=False)
r_al = dr.reduceRM(r_al)

0.0712471111111


In [28]:
#track-artist
icm = dr.getICM_csr(albums=False,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
s.setdiag(0)
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)
r_ar = dr.reduceRM(r_ar)

0.0702706944444


In [4]:
#album and artist
icm = dr.getICM_csr(albums=True,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
r_al_ar = urm*s.T
r_al_ar = normalize(r_al_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_al_ar,verbose=False)
r_al_ar = dr.reduceRM(r_al_ar)

In [5]:
#tags
icm = dr.getICM_csr(albums=False,artists=False,tags=True)
#icm = normalize(icm,axis=1)   # little improvment, no more
model = BM25Recommender(K = 45)
model.fit(icm)
s = model.similarity
s.setdiag(0)
r_tag = urm*s.T
r_tag = normalize(r_tag, axis=0)
if evaluation: print dr.evaluateMAP(r_tag, verbose = False)
r_tag = dr.reduceRM(r_tag)

In [6]:
#cf ub
model = BM25Recommender(K=60)
model.fit(urm)
s = model.similarity
r_cfub = s.T * urm
r_cfub = normalize(r_cfub, axis=1)
if evaluation: print dr.evaluateMAP(r_cfub, verbose = False)
r_cfub = dr.reduceRM(r_cfub)

In [5]:
#cf ib
model = BM25Recommender(K=100)
model.fit(urm.T)
s = model.similarity
#s.setdiag(0)
r_cfib = urm * s.T
r_cfib = normalize(r_cfib, axis=0)
if evaluation: print dr.evaluateMAP(r_cfib, verbose = False)
r_cfib = dr.reduceRM(r_cfib)

0.0729845


In [8]:
#owners
ucm = dr.getOwnersUCM_csr()
ucm = normalize(ucm,axis=0)
model = BM25Recommender(K=65)
model.fit(ucm)
s = model.similarity
r_o = s*urm
r_o = normalize(r_o, axis=0)
if evaluation: print dr.evaluateMAP(r_o,verbose=False)
r_o = dr.reduceRM(r_o)

In [11]:
############# start with the mix #################

In [12]:
########## cb

In [23]:
r_cb1 = r_al_ar + 0.24*r_tag 
if evaluation: print dr.evaluateMAP(r_cb1, verbose = False)

0.0809853055556


In [25]:
########## cf

In [26]:
#total cf
r_totcf = 1.8*r_cfib + r_cfub
if evaluation: print dr.evaluateMAP(r_totcf, verbose = False)

0.0781461944444


In [27]:
############# mix all and tune parameters #################

In [28]:
r_tot = r_cb1 + 1.80 * r_totcf
if evaluation: print dr.evaluateMAP(r_tot,verbose = False)

0.102670138889


In [20]:
######improve total a little

In [21]:
r_tot2 = r_tot + 0.25*r_pal + 0.25*r_par
if evaluation: print dr.evaluateMAP(r_tot2,verbose = False)

0.104498


In [29]:
r_tot3 = r_tot + 0.10*r_al + 0.10*r_ar    #also rtot2
if evaluation: print dr.evaluateMAP(r_tot3,verbose = False)

0.103384916667


In [ ]:
############################### Formula best score

In [10]:
r_tot5 = 1.809 * 1.250 * r_cfib + 1.820 * r_cfub + 1*r_al_ar + 0.19 * r_tag + 0.150*r_al + 0.119*r_ar
if evaluation: print dr.evaluateMAP(r_tot5,verbose = False)

In [9]:
############################### Project X

In [15]:
r_totx = 1.80 * 1.250 * r_cfib + 1.820 * r_cfub + 1.02*r_al_ar + 0.19* r_tag + 0.750 * r_o + 0.150*r_al + 0.119*r_ar
if evaluation: print dr.evaluateMAP(r_totx,verbose = False)

In [10]:
###########care this solution use the tags without remove diagonal
#r_totx = 1.809 * 1.250 * r_cfib + 1.820 * r_cfub + r_al_ar + 0.240 * r_tag + 0.751 * r_o + 0.150*r_al + 0.119*r_ar
#if evaluation: print dr.evaluateMAP(r_totx,verbose = False)

In [ ]:
##############################

In [16]:
rec = dr.getAllRecommendations(r_totx)
if not evaluation: dr.buildSubmissionFile(recommendation=rec, file_name="a natale puoi boostarlo anche tuuuu 18") ####<------name

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommendations done
Creating submission file in: ./Submissions/a natale puoi boostarlo anche tuuuu 18
Done


In [ ]:
#############################

In [10]:
r_bpr = dr.reduceRM(urm * dr.load_sparse_csr(filesslim))
if evaluation: print dr.evaluateMAP(r_bpr, verbose=False)

In [ ]:
#tuning
for p in np.arange(0.5,1.5,0.02):
    r_totx = 1.809 * 1.250 * r_cfib + 1.820 * r_cfub + 1*r_al_ar + 0.19* r_tag + p * r_o + 0.150*r_al + 0.119*r_ar
    score = dr.evaluateMAP(r_totx,verbose = False)
    print ("k = {:1.2f} \t\tmap = {:1.5f}").format(p,score)

In [19]:
#not used atm
#r_tot2 = r_tot + 5* r_bpr

#if evaluation: print dr.evaluateMAP(r_tot2,verbose = False)

In [11]:
#playtlist-album
icm_al = dr.getUCM_csr(albums=True,artists=False)
model = BM25Recommender(K=100)
model.fit(icm_al)
s = model.similarity
s.setdiag(0)
r_pal = s*urm
r_pal = normalize(r_pal, axis=0)
if evaluation: print dr.evaluateMAP(r_pal,verbose=False)
r_pal = dr.reduceRM(r_pal)

In [5]:
#playlist-artist
icm_ar = dr.getUCM_csr(albums=False,artists=True)
model = BM25Recommender(K=100)
model.fit(icm_ar)
s = model.similarity
r_par = s.T*urm
r_par = normalize(r_par, axis=0)
if evaluation: print dr.evaluateMAP(r_par,verbose=False)
r_par = dr.reduceRM(r_par)